In [12]:
from langgraph.graph import END, StateGraph
from langchain_core.runnables import RunnablePassthrough

passthrough_node = RunnablePassthrough()

def path1(state: dict):
    print("
==>> find_relevant_report")
    documents = state["documents"]
    documents.append("path1")

    return {"documents": documents}


def path2(state: list):
    print("
==>> find_to_update")
    documents = state["documents"]
    documents.append("path2")

    return {"documents": documents}


def middle_router(state: dict):
    print("
==>> middle_router")

    return ["path1", "path2"]


def rendezvous(state: dict):
    print("
==>> add_message_from_state_to_documents")
    documents = state["documents"]
    print("documents: ", documents)
    return {"documents": documents}

In [13]:
g = StateGraph({"documents": []})

g.add_node("start", passthrough_node)
g.add_conditional_edges(
    "start",
    middle_router,
    {
        "path1": "path1",
        "path2": "path2",
    },
    then="rendezvous",
)

g.add_node("path1", path1)
g.add_edge("path1", "rendezvous")

g.add_node("path2", path2)
g.add_edge("path2", "rendezvous")

g.add_node("rendezvous", rendezvous)
g.add_edge("rendezvous", END)

g.set_entry_point("start")
langgraph_app = g.compile()

langgraph_app.invoke({"documents": []})

LangGraph Input:  {'documents': []}
LastValue.update []
LastValue.update [{'documents': []}]
LastValue.update [{'documents': []}]

==>> middle_router
LastValue.update [{'documents': []}]

==>> find_relevant_report

==>> find_to_update
LastValue.update [{'documents': ['path1', 'path2']}, {'documents': ['path1', 'path2']}]


InvalidUpdateError: Invalid update for channel __root__: LastValue can only receive one value per step.

In [16]:
from langgraph.graph import END, StateGraph
from langchain_core.runnables import RunnablePassthrough
from langgraph.graph import StateGraph
from typing_extensions import TypedDict
from typing import Annotated
import operator


class State(TypedDict):
    aggregate: Annotated[list, operator.add]
    which: str


def path1(state: dict):
    return {"aggregate": ["path1"]}


def path2(state: dict):
    return {"aggregate": ["path2"]}


def middle_router(state: dict):
    print("
==>> middle_router")

    return ["path1", "path2"]


def rendezvous(state: dict):
    print("
==>> rendezvous")
    return 


g = StateGraph(State)

g.add_node("start", RunnablePassthrough())
g.set_entry_point("start")

g.add_conditional_edges(
    "start",
    middle_router,
    {
        "path1": "path1",
        "path2": "path2",
    },
    then="rendezvous",
)

g.add_node("path1", path1)
g.add_edge("path1", "rendezvous")

g.add_node("path2", path2)
g.add_edge("path2", "rendezvous")

g.add_node("rendezvous", rendezvous)
g.add_edge("rendezvous", END)

langgraph_app = g.compile()

# langgraph_app.invoke({"aggregate": []})


def var_name(var):
    for name, value in globals().items():
        if value is var:
            return name
        

var_name(langgraph_app)

'langgraph_app'

In [20]:
from langgraph.graph import END, StateGraph
from langchain_core.runnables import RunnablePassthrough



def simp(state: dict):
    return "ask_name"

def ask_name(state: dict):
    print("
==>> ask_name")
    return

def greeting(state: dict):
    print("
==>> greeting")
    return

g = StateGraph({"documents": []})
g.set_entry_point("entry")

g.add_node("entry", RunnablePassthrough())
g.add_conditional_edges(
    "entry",
    simp,
    {
        "ask_name": "ask_name",
        "greeting": "greeting",
    },
    then=END,
)

g.add_node("ask_name", ask_name)
g.add_node("greeting", greeting)

start_of_chat = g.compile()
var_name(start_of_chat)

'start_of_chat'

In [3]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.base import RunnableParallel


def add_one(x: int) -> int:
    return x + 1


def mul_two(x: int) -> int:
    return x * 2


def mul_three(x: int) -> int:
    return x * 3


runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)

sequence = runnable_1 | {  # this dict is coerced to a RunnableParallel
    "mul_two": runnable_2,
    "mul_three": runnable_3,
}
# Or equivalently:
# sequence = runnable_1 | RunnableParallel(
#     {"mul_two": runnable_2, "mul_three": runnable_3}
# )
# Also equivalently:
sequence = runnable_1 | RunnableParallel(
    mul_two=runnable_2,
    mul_three=runnable_3,
)

sequence.invoke(1)
await sequence.ainvoke(1)

# sequence.batch([1, 2, 3])
# await sequence.abatch([1, 2, 3])

{'mul_two': 4, 'mul_three': 6}